# Capítulo 05: Time Travel e Versionamento

Este notebook demonstra o uso de Time Travel para consultar versões históricas de dados.

**Objetivo:** Dominar consultas temporais e auditoria de mudanças.

**Conteúdo:**
- Time Travel básico
- Comparação entre versões
- Recuperação de dados deletados
- Análises históricas
- Boas práticas de performance

## 8. Resumo

Neste capítulo você aprendeu:
- ✅ Como usar Time Travel para consultar dados históricos
- ✅ Comparar versões e auditar mudanças
- ✅ Recuperar dados deletados
- ✅ Fazer análises históricas mensais
- ✅ Boas práticas de performance em Time Travel

**Parabéns!** Você completou os fundamentos de DuckDB + Iceberg!

In [ ]:
# ✅ BOM: Especificar colunas necessárias
query_bom = """
SELECT customer_id, amount
FROM iceberg_scan(
    's3://bucket/sales',
    snapshot_from_timestamp = '2024-01-01'::TIMESTAMP
)
"""

# ❌ RUIM: SELECT * em snapshot antigo (mais lento)
query_ruim = """
SELECT *
FROM iceberg_scan(
    's3://bucket/sales',
    snapshot_from_timestamp = '2024-01-01'::TIMESTAMP
)
"""

print("Boas Práticas:")
print("✅ Selecione apenas colunas necessárias")
print("✅ Use filtros para reduzir dados lidos")
print("✅ Cache snapshots frequentemente consultados em tabela temporária")
print("\nExemplo de cache:")
print("""
CREATE TEMP TABLE snapshot_jan1 AS
SELECT *
FROM iceberg_scan('s3://bucket/sales', snapshot_from_timestamp = '2024-01-01'::TIMESTAMP)
""")

## 7. Boas Práticas de Performance

In [ ]:
class IcebergVersionManager:
    """Gerencia versões de tabelas Iceberg"""
    
    def __init__(self, table_path):
        self.table_path = table_path
        self.con = duckdb.connect()
        safe_install_ext(self.con, "iceberg")
    
    def list_versions(self):
        """Lista todas as versões disponíveis"""
        try:
            return self.con.execute(f"""
                SELECT
                    snapshot_id,
                    sequence_number,
                    to_timestamp(timestamp_ms / 1000) as created_at,
                    timestamp_ms
                FROM iceberg_snapshots('{self.table_path}')
                ORDER BY sequence_number DESC
            """).df()
        except Exception as e:
            print(f"❌ Erro: {e}")
            return None
    
    def get_version_at_time(self, timestamp):
        """
        Retorna snapshot mais próximo de um timestamp
        
        Args:
            timestamp: datetime object
        """
        try:
            result = self.con.execute(f"""
                SELECT snapshot_id
                FROM iceberg_snapshots('{self.table_path}')
                WHERE timestamp_ms <= {int(timestamp.timestamp() * 1000)}
                ORDER BY sequence_number DESC
                LIMIT 1
            """).fetchone()
            
            return result[0] if result else None
        except Exception as e:
            print(f"❌ Erro: {e}")
            return None

# Exemplo de uso
# vm = IcebergVersionManager('s3://bucket/sales')
# versions = vm.list_versions()
print("✅ Classe IcebergVersionManager criada!")

## 6. Classe Version Manager

In [ ]:
def monthly_snapshot_analysis(table_path, months_back=12):
    """
    Análise de snapshots mensais
    
    Args:
        table_path: Caminho da tabela Iceberg
        months_back: Número de meses para analisar
    
    Returns:
        DataFrame com estatísticas mensais
    """
    results = []
    
    for i in range(months_back):
        month_date = datetime.now() - timedelta(days=30 * i)
        month_str = month_date.strftime('%Y-%m-01 00:00:00')
        
        try:
            stats = con.execute(f"""
                SELECT
                    '{month_str}' as snapshot_date,
                    count(*) as record_count,
                    sum(amount) as total_amount
                FROM iceberg_scan(
                    '{table_path}',
                    snapshot_from_timestamp = '{month_str}'::TIMESTAMP
                )
            """).fetchone()
            
            results.append(stats)
        except Exception as e:
            print(f"⚠️  Snapshot {month_str} não disponível")
            continue
    
    if results:
        df = pd.DataFrame(results, columns=['date', 'records', 'amount'])
        print(f"✅ Análise de {len(results)} meses concluída")
        return df
    else:
        print("❌ Nenhum snapshot encontrado")
        return None

# Exemplo: history = monthly_snapshot_analysis('s3://bucket/sales', months_back=12)
print("✅ Função monthly_snapshot_analysis() criada!")

## 5. Análise Histórica Mensal

In [ ]:
def recover_deleted_records(table_path, before_deletion_timestamp):
    """
    Recupera registros que foram deletados
    
    Args:
        table_path: Caminho da tabela Iceberg
        before_deletion_timestamp: Momento antes da deleção
    
    Returns:
        DataFrame com registros deletados
    """
    try:
        # Dados antes da deleção
        before = con.execute(f"""
            SELECT *
            FROM iceberg_scan(
                '{table_path}',
                snapshot_from_timestamp = '{before_deletion_timestamp}'::TIMESTAMP
            )
        """).df()
        
        # Dados atuais
        current = con.execute(f"""
            SELECT *
            FROM iceberg_scan('{table_path}')
        """).df()
        
        # Encontrar deletados (assumindo coluna 'id')
        deleted = before[~before['id'].isin(current['id'])]
        
        print(f"✅ {len(deleted)} registros deletados encontrados")
        return deleted
    except Exception as e:
        print(f"❌ Erro: {e}")
        return None

# Exemplo: deleted_records = recover_deleted_records('s3://bucket/users', '2024-01-15 10:00:00')
print("✅ Função recover_deleted_records() criada!")

## 4. Recuperação de Dados Deletados

In [ ]:
def audit_changes(table_path, timestamp1, timestamp2):
    """
    Compara dados entre dois momentos
    
    Args:
        table_path: Caminho da tabela Iceberg
        timestamp1: Primeiro momento (formato: 'YYYY-MM-DD HH:MM:SS')
        timestamp2: Segundo momento (formato: 'YYYY-MM-DD HH:MM:SS')
    
    Returns:
        DataFrame com mudanças detectadas
    """
    # Dados no tempo 1
    t1 = con.execute(f"""
        SELECT customer_id, sum(amount) as total
        FROM iceberg_scan(
            '{table_path}',
            snapshot_from_timestamp = '{timestamp1}'::TIMESTAMP
        )
        GROUP BY customer_id
    """).df()
    
    # Dados no tempo 2
    t2 = con.execute(f"""
        SELECT customer_id, sum(amount) as total
        FROM iceberg_scan(
            '{table_path}',
            snapshot_from_timestamp = '{timestamp2}'::TIMESTAMP
        )
        GROUP BY customer_id
    """).df()
    
    # Comparar
    merged = t1.merge(t2, on='customer_id', how='outer', suffixes=('_t1', '_t2'))
    merged['change'] = merged['total_t2'] - merged['total_t1']
    
    return merged[merged['change'] != 0]

# Exemplo de uso
# changes = audit_changes('s3://bucket/sales', '2024-01-01 00:00:00', '2024-01-31 23:59:59')
print("✅ Função audit_changes() criada!")

## 3. Auditoria de Mudanças

Função para identificar mudanças entre dois momentos.

In [ ]:
# Exemplo: Comparar dados de hoje vs semana passada
week_ago = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d %H:%M:%S')

query_current = """
SELECT count(*) as total, sum(amount) as revenue
FROM iceberg_scan('s3://bucket/sales')
"""

query_past = f"""
SELECT count(*) as total, sum(amount) as revenue
FROM iceberg_scan(
    's3://bucket/sales',
    snapshot_from_timestamp = '{week_ago}'::TIMESTAMP
)
"""

print("Query para dados atuais:")
print(query_current)
print("\nQuery para dados históricos (7 dias atrás):")
print(query_past)
print("\nℹ️  O parâmetro 'snapshot_from_timestamp' permite Time Travel")

## 2. Time Travel Básico

Comparar dados entre diferentes momentos no tempo.

In [ ]:
# Importar bibliotecas
import duckdb
import pandas as pd
from datetime import datetime, timedelta
import importlib.util


def safe_install_ext(con, ext_name):
    try:
        con.execute(f"INSTALL {ext_name}")
        con.execute(f"LOAD {ext_name}")
        print(f"✅ {ext_name} carregado")
        return True
    except Exception as e:
        print(f"⚠️  Erro: {e}")
        return False


# Setup
con = duckdb.connect()
safe_install_ext(con, "iceberg")
safe_install_ext(con, "httpfs")
print("✅ Ambiente configurado!")

In [ ]:
# Instalar dependências
import sys
!{sys.executable} -m pip install -q duckdb pyarrow pandas

## 1. Setup Inicial